### 1. Downloading the dataset from kaggle

In [ ]:
pip install kaggle

In [ ]:
# Upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rachnadevraj","key":"0b385af78d7dc40bfdec644e2566fd6c"}'}

In [ ]:
# Secure the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle datasets download -d kmkarakaya/logos-bk-kfc-mcdonald-starbucks-subway-none

Dataset URL: https://www.kaggle.com/datasets/kmkarakaya/logos-bk-kfc-mcdonald-starbucks-subway-none
License(s): DbCL-1.0
 68% 44.0M/65.0M [00:00<00:00, 117MB/s]
100% 65.0M/65.0M [00:00<00:00, 144MB/s]


In [ ]:
!unzip logos-bk-kfc-mcdonald-starbucks-subway-none.zip -d dataset/

Archive:  logos-bk-kfc-mcdonald-starbucks-subway-none.zip
  inflating: dataset/logos3/test/Burger King/armada_image_100.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_100_2.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_101.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_102.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_103.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_3.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_4.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_444.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_445.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_446.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_447.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_448.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_5.jpg  
  inflating: dataset/logos3/test/Burger King/armada_image_534.

In [ ]:
import os
print(os.listdir("dataset/"))

['logos_v3_mini', 'logos3']


In [ ]:
print(os.listdir("dataset/logos_v3_mini/logos3"))

['test', 'train']


### 2 & 3. Preprocess the data(resize the images) and split into train and test & Load a pre-trained model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import os

# Set random seed for reproducibility
torch.manual_seed(2)
torch.cuda.manual_seed_all(2)

# Define dataset path
dataset_path = "dataset/logos_v3_mini/logos3"

# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match ResNet input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load datasets
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")

full_train_ds = datasets.ImageFolder(root=train_dir, transform=transform)
test_ds = datasets.ImageFolder(root=test_dir, transform=transform)

# Split train dataset into 80% train, 20% validation
train_size = int(0.8 * len(full_train_ds))
val_size = len(full_train_ds) - train_size
train_ds, val_ds = random_split(full_train_ds, [train_size, val_size])

# Define batch size
batch_size = 32

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Load Pretrained ResNet model
resnet = models.resnet18(pretrained=True)

# Modify final layer to match number of classes
num_classes = len(full_train_ds.classes)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training Loop
num_epochs = 1
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    resnet.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = resnet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}")

# Save Model
torch.save(resnet.state_dict(), "resnet_model.pth")

print("Training completed and model saved!")


Epoch 1/1, Loss: 0.2031, Val Loss: 0.3133, Val Acc: 0.9368
Training completed and model saved!


### 4 & 5. Replace last fully connected layer & Freeze the weights of pre-trained layers and train only new fully connected layer

In [ ]:
# Updating the number of neurons in the output layer to 6 - based on the number of classes in the dataset.
# Freezing the weights of all other layers and training only the last layer

# Load Pretrained ResNet Model
resnet = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

# Modify the Output Layer
num_classes = 6  # Since we have 6 classes in our dataset
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)  # Replace last layer

# Define Transformations for Training & Testing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Dataset from Folder
data_dir = "dataset/logos_v3_mini/logos3/train"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split into Training & Validation (80:20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create Dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)  # Only updating last layer

# Training Loop
num_epochs = 1
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

# Save Model
torch.save(resnet.state_dict(), "resnet_finetuned.pth")
print("Training Complete & Model Saved!")

Epoch 1, Loss: 1.1469, Train Accuracy: 60.72%
Training Complete & Model Saved!


### 6. Evaluate the performance of the model

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Set model to evaluation mode
resnet.eval()

# Lists to store predictions & actual labels
all_preds = []
all_labels = []

# Disable gradient calculations
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = resnet(images)  # Get model predictions
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability

        all_preds.extend(predicted.cpu().numpy())  # Convert to numpy & store
        all_labels.extend(labels.cpu().numpy())

# Compute Performance Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

# Print Results
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Validation Accuracy: 81.32%
Precision: 0.8223
Recall: 0.8132
F1-score: 0.7933


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 7. Fine tune the entire model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Unfreeze the last few layers of ResNet for fine-tuning
for param in resnet.parameters():
    param.requires_grad = False  # Keep layers frozen

# Unfreeze last 2 ResNet layers
for param in list(resnet.layer4.parameters()):
    param.requires_grad = True  # Allow training

# Define new fully connected layer (6 classes)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 6)  # Output layer with 6 neurons

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': resnet.layer4.parameters(), 'lr': 1e-4},  # Lower LR for fine-tuning
    {'params': resnet.fc.parameters(), 'lr': 1e-3}  # Higher LR for new FC layer
])

# Train the fine-tuned model
num_epochs = 1  # Adjust based on dataset size
for epoch in range(num_epochs):
    resnet.train()  # Set to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients
        outputs = resnet(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Fine-tuning complete!")


Epoch [1/1], Loss: 0.4117
Fine-tuning complete!


### 8. Again evaluate the performance

In [ ]:
from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(train_dataset))  # 80% training
val_size = len(train_dataset) - train_size  # 20% validation

# Split the dataset
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import torch.nn.functional as F

# Move model to evaluation mode
resnet.eval()

# Initialize variables
correct = 0
total = 0
val_loss = 0.0

# Disable gradient calculation
with torch.no_grad():
    for images, labels in validation_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        # Get predictions
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Compute average loss and accuracy
avg_loss = val_loss / len(validation_loader)
accuracy = 100 * correct / total

# Print results
print(f"Validation Loss: {avg_loss:.4f}")
print(f"Validation Accuracy: {accuracy:.2f}%")


Validation Loss: 0.0237
Validation Accuracy: 100.00%
